In [3]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import models, datasets
import torchvision.transforms as transforms
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

class notMNIST(Dataset):
    def __init__(self, data, labels):
        self.images = data
        self.labels = labels

        self.transformation = transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        img = self.images[index]
        label = self.labels[index]
        img_tensor = self.transformation(img)
        y_tensor = torch.tensor(label, dtype=torch.long)
        return img_tensor.view(1, 28, 28), y_tensor

data = loadmat('/home/adambucko/workspace/hns/zadanie1_pismenka/notMNIST_small.mat')
print(data.keys())

images = data['images']
labels = data['labels']

images = [images[:, :, i] for i in range(0, images.shape[2])]
images = np.asarray(images)

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.4, shuffle=True)

train_dataset = notMNIST(x_train, y_train)
test_dataset = notMNIST(x_test, y_test)

dict_keys(['__header__', '__version__', '__globals__', 'images', 'labels'])


In [4]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()

        # Input layer: 28*28 neurons (for each pixel in the image)
        # First hidden layer: 512 neurons
        self.fc1 = nn.Linear(28*28, 512)
        
        # Second hidden layer: 256 neurons
        self.fc2 = nn.Linear(512, 256)
        
        # Third hidden layer: 128 neurons
        self.fc3 = nn.Linear(256, 128)
        
        # Output layer: 10 neurons (one for each class A-J)
        self.fc4 = nn.Linear(128, 10)
        
        # Activation function: ReLU
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input tensor
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # First convolutional layer: 1 input channel, 32 output channels
        # Kernel size: 3x3
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling with 2x2 window
        
        # Second convolutional layer: 32 input channels, 64 output channels
        # Kernel size: 3x3
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        # Fully connected layer: 64*7*7 input neurons (from the last convolutional layer)
        # 128 output neurons
        self.fc1 = nn.Linear(64*7*7, 128)
        
        # Output layer: 128 input neurons, 10 output neurons (one for each class A-J)
        self.fc2 = nn.Linear(128, 10)
        
        # Activation function: ReLU
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64*7*7)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [6]:
# initialize the models
mlp_model = MLP()
cnn_model = CNN()


In [7]:
# define the loss functions and optimizers
criterion = nn.CrossEntropyLoss()

mlp_optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001)
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

In [8]:
# Training
def train_model(model, optimizer, criterion, dataloader, epochs=10):
    """
    Train a given model using the specified optimizer and criterion.

    Args:
    - model: PyTorch model to be trained
    - optimizer: Optimizer for updating model parameters
    - criterion: Loss function
    - dataloader: DataLoader for the training data
    - epochs: Number of epochs (default=10)

    Returns:
    - list of losses per epoch
    """

    model.train()  # Set the model to training mode
    loss_list = []  # To store the loss per epoch

    # Loop over epochs
    for epoch in trange(epochs, desc="Epochs"):
        epoch_loss = 0.0  # Accumulator for the loss

        # Loop over batches
        for inputs, labels in tqdm(dataloader, desc="Batches", leave=False):
            optimizer.zero_grad()  # Clear gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        loss_list.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    return loss_list


In [9]:
def evaluate_model(model, dataloader):
    """
    Evaluate the model's performance on the given dataset.

    Args:
    - model: Trained PyTorch model
    - dataloader: DataLoader for the evaluation data

    Returns:
    - Accuracy
    """

    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to track gradients during evaluation
        for inputs, labels in tqdm(dataloader, desc="Evaluation", leave=False):
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


In [ ]:
# using creating functions and classes to train models

# Training the MLP model
mlp_losses = train_model(mlp_model, mlp_optimizer, criterion, DataLoader(train_dataset, batch_size=64, shuffle=True), epochs=10)

# Evaluating the MLP model
mlp_accuracy = evaluate_model(mlp_model, DataLoader(test_dataset, batch_size=64))
print(f"MLP Model Accuracy: {mlp_accuracy:.2f}%")

# Training the CNN model
cnn_losses = train_model(cnn_model, cnn_optimizer, criterion, DataLoader(train_dataset, batch_size=64, shuffle=True), epochs=10)

# Evaluating the CNN model
cnn_accuracy = evaluate_model(cnn_model, DataLoader(test_dataset, batch_size=64))
print(f"CNN Model Accuracy: {cnn_accuracy:.2f}%")
